In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

In [2]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

In [3]:
from fastai.text.all import *

In [4]:
path = untar_data(URLs.IMDB)
path.ls()


(#7) [Path('/home/advaitmb/.fastai/data/imdb/test'),Path('/home/advaitmb/.fastai/data/imdb/train'),Path('/home/advaitmb/.fastai/data/imdb/imdb.vocab'),Path('/home/advaitmb/.fastai/data/imdb/tmp_clas'),Path('/home/advaitmb/.fastai/data/imdb/README'),Path('/home/advaitmb/.fastai/data/imdb/unsup'),Path('/home/advaitmb/.fastai/data/imdb/tmp_lm')]

In [4]:
path = '/home/advaitmb/.fastai/data/imdb/train/pos/'

In [24]:
len(path.ls())

AttributeError: 'str' object has no attribute 'ls'

In [7]:
positive_train_path = '/home/advaitmb/.fastai/data/imdb/train/pos/'
positive_train_files = get_text_files(positive_train_path)

negative_train_path = '/home/advaitmb/.fastai/data/imdb/train/neg/'
negative_train_files = get_text_files(negative_train_path)


positive_test_path = '/home/advaitmb/.fastai/data/imdb/test/pos/'
positive_test_files = get_text_files(positive_test_path)

negative_test_path = '/home/advaitmb/.fastai/data/imdb/test/neg/'
negative_test_files = get_text_files(negative_test_path)

In [9]:
positive_test_files

(#12500) [Path('/home/advaitmb/.fastai/data/imdb/test/pos/364_10.txt'),Path('/home/advaitmb/.fastai/data/imdb/test/pos/11789_9.txt'),Path('/home/advaitmb/.fastai/data/imdb/test/pos/10290_10.txt'),Path('/home/advaitmb/.fastai/data/imdb/test/pos/1447_9.txt'),Path('/home/advaitmb/.fastai/data/imdb/test/pos/148_9.txt'),Path('/home/advaitmb/.fastai/data/imdb/test/pos/9414_10.txt'),Path('/home/advaitmb/.fastai/data/imdb/test/pos/5983_10.txt'),Path('/home/advaitmb/.fastai/data/imdb/test/pos/4981_8.txt'),Path('/home/advaitmb/.fastai/data/imdb/test/pos/10981_9.txt'),Path('/home/advaitmb/.fastai/data/imdb/test/pos/3932_8.txt')...]

In [10]:
def files_to_list(files):
    ls = []
    for file in files:
        with open(file, 'r') as f:
            ls.append(f.read())
    return ls


In [11]:
positive_train_text_list = files_to_list(positive_train_files)
negative_train_text_list = files_to_list(negative_train_files)
positive_test_text_list = files_to_list(positive_test_files)
negative_test_text_list = files_to_list(negative_test_files)

In [12]:
len_pos_train = len(positive_train_text_list); print(len_pos_train)

12500


In [13]:
all_pos_text = all_texts = np.concatenate([np.array(positive_train_text_list), np.array(positive_test_text_list)])
all_pos_text.shape

(25000,)

In [20]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [30]:
for i in range(len(all_pos_text)):
    all_pos_text[i] = cleanhtml(all_pos_text[i])

In [31]:
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [32]:
splits = [range_of(len_pos_train), list(range(len_pos_train, len(all_pos_text)))]
tls = TfmdLists(all_pos_text, TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)

In [38]:
bs,sl = 2,1024
dls = tls.dataloaders(bs=bs, seq_len=sl)

In [34]:
dls.show_batch(max_n=2)

,text,text_
0,"Eddie Murphy Delirious is by far the funniest thing you will ever see in your life. You can compare it to any movie, and I garuntee you will decide that Delirious is the funniest movie ever! This movie is about 1hr. 45 mins., and throughout that time, there was barely a moment where I wasn't laughing. You will laugh for hours after it is over, replaying the punch lines over and over and over in your head. Eddie Murphy has given so many funny performances over his career (48 Hrs.,Trading Places,Beverly Hills Cop,Raw,Coming To America, The Nutty professor,Shrek,etc.),but this is by far his MOST HILARIOUS moment. I have seen this movie so many times, and it is funnier every time. It never loses its edge. From this day forward, every great stand up performance will be emulated from Delirious. ***** and two thumbs up!I spotted in the guide to films","ddie Murphy Delirious is by far the funniest thing you will ever see in your life. You can compare it to any movie, and I garuntee you will decide that Delirious is the funniest movie ever! This movie is about 1hr. 45 mins., and throughout that time, there was barely a moment where I wasn't laughing. You will laugh for hours after it is over, replaying the punch lines over and over and over in your head. Eddie Murphy has given so many funny performances over his career (48 Hrs.,Trading Places,Beverly Hills Cop,Raw,Coming To America, The Nutty professor,Shrek,etc.),but this is by far his MOST HILARIOUS moment. I have seen this movie so many times, and it is funnier every time. It never loses its edge. From this day forward, every great stand up performance will be emulated from Delirious. ***** and two thumbs up!I spotted in the guide to films"
1,"Or the clever use of photography to capture Hayworth diving from a rock, reflected on the lecherous Grisby's binoculars.The acting is pretty good, in particular Sloane (possibly that fine actor's best film role). Glenn Anders was a leading Broadway performer. He rarely made movies before THE LADY FROM SHANGHAI, and his slimy Grisby is unforgettable. Also Ted De Corsia does very nicely with Broome - a detective who is really looking for his own interests, to his own cost.As for Hayworth, she turns in a performance that was unlike most of what she had done before (BLOOD AND SAND, TALES OF MANHATTAN, and THE STRAWBERRY BLOND are exceptions), and is a memorable siren. Welles' O'Hara is a very unusual character for the actor - a likable but naive man who learns the hard way not to believe what he secretly wants to believe. It's not KANE, AMBERSOMS, OTHELLO, TOUCH","the clever use of photography to capture Hayworth diving from a rock, reflected on the lecherous Grisby's binoculars.The acting is pretty good, in particular Sloane (possibly that fine actor's best film role). Glenn Anders was a leading Broadway performer. He rarely made movies before THE LADY FROM SHANGHAI, and his slimy Grisby is unforgettable. Also Ted De Corsia does very nicely with Broome - a detective who is really looking for his own interests, to his own cost.As for Hayworth, she turns in a performance that was unlike most of what she had done before (BLOOD AND SAND, TALES OF MANHATTAN, and THE STRAWBERRY BLOND are exceptions), and is a memorable siren. Welles' O'Hara is a very unusual character for the actor - a likable but naive man who learns the hard way not to believe what he secretly wants to believe. It's not KANE, AMBERSOMS, OTHELLO, TOUCH OF"


In [35]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [36]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

In [44]:
learn.validate()

(#2) [3.657973051071167,38.78265380859375]

In [37]:
learn.lr_find()

RuntimeError: CUDA out of memory. Tried to allocate 148.00 MiB (GPU 0; 11.17 GiB total capacity; 6.68 GiB already allocated; 37.94 MiB free; 6.80 GiB reserved in total by PyTorch)